# Assignment 1
> Reinforcement Learning Programming- CSCN 8020 <br>
> Name: Tai Siang Huang <br>
> ID: 9006413 

## **Problem 1 [10]**
Pick-and-Place Robot: Consider using reinforcement learning to control the motion of a robot arm in a repetitive pick-and-place task.
Design the reinforcement learning problem as an MDP, define states, actions, rewards with reasoning.

### Assuming we are controlling a 3-DOF robotic arm

### Sates
1. **Joint Angles:** `[θ₁, θ₂, θ₃]` <br>
   - Reason: Agent can learn the relationship between itself joint anglesa and space position.<br>
2. **Joint Angles' speed:** `[θ₁_v, θ₂_v, θ₃_v]`<br>
   - Reason: Agent can learn for controlling smooth and fast movement.<br>
3. **End-effector Position:** `[x, y, z]`<br>
   - Reason: Agent can determine the space's position itself.<br>
4. **Graspper State:** `[gripper_state]`<br>
   - Reason: Agent can know the gripper status.<br>
5. **Object Position:** `[x_obj, y_obj, z_obj]`<br>
   - Reason: Agent can learn where the object is and where to catch <br>
6. **Target Position:** `[x_target, y_target, z_target]`<br>
   - Reason: Agent can learn where the target is and where to place<br>
7. **Step Count:** `[step_count]`<br>
   - Reason: Agent can learn how to complete the task in the shortest time<br>

**Total Sates:** `s = [θ₁, θ₂, θ₃, θ₁_v, θ₂_v, θ₃_v, x, y, z, gripper_state, x_obj, y_obj, z_obj, x_target, y_target, z_target, step_count]`


### Actions
- `a = [Δx, Δy, Δz, g]`
- **End-effector Control:** Learning policies is more intuitive, "move towards an object" rather than "adjust a joint"
- **Gripper Action:** Gripper behavior: 1 = off, 0 = on

### Rewards
1. **Close to object:** Encourage the agent to move closer to the object.
2. **Successfully grasp the object:** A large reward will be given when the object is successfully grasped.
3. **Close to the target location:** When the object is grasped, encourage approach to the target location.
4. **Successfully drop the object in the target area:** Big rewards for completing tasks.
5. **Additional penalties:** Encourage completing tasks faster by penalizing every step, penalty for placing or dropping in the wrong place.
   
**Note:** Moving tasks use the linear positive to give reward, can control robot smooth.

In [ ]:
# ------------------------------
# Reward Function
# ------------------------------
def compute_reward(state):
    ee_pos = np.array(state["end_effector_pos"])
    obj_pos = np.array(state["object_pos"])
    tgt_pos = np.array(state["target_pos"])
    
    # Tunable parameters
    grasp_threshold = 0.05      # If EE-object distance < this, consider grasped
    place_threshold = 0.05      # If object-target distance < this, consider placed
    drop_threshold = 0.1        # If object far from both EE and target, consider dropped
    max_distance = 1.0          # For distance normalization

    # Distance calculations
    dist_to_object = np.linalg.norm(ee_pos - obj_pos)
    dist_to_target = np.linalg.norm(obj_pos - tgt_pos)
    
    # State inference
    grasped = state["gripper_state"] == 1 and dist_to_object < grasp_threshold
    placed_correctly = state["gripper_state"] == 0 and dist_to_target < place_threshold
    dropped_object_elsewhere = (
        state["gripper_state"] == 0 and
        dist_to_object > drop_threshold and
        dist_to_target > drop_threshold
    )
    
    # ------------------------------
    # Reward calculation (linear distance + task stages)
    r1 = 1 - (dist_to_object / max_distance)        # Closer to object
    r1 = max(r1, 0)
    
    r2 = 10 if grasped else 0                       # Successful grasp
    
    r3 = 0
    if grasped:
        ee_to_target = np.linalg.norm(ee_pos - tgt_pos)
        r3 = 1 - (ee_to_target / max_distance)      # While grasping, closer to target
        r3 = max(r3, 0)
    
    r4 = 20 if placed_correctly else 0              # Successful placement
    r5 = -0.1                                       # Step penalty
    r6 = -10 if dropped_object_elsewhere else 0     # Drop penalty

    total_reward = r1 + r2 + r3 + r4 + r5 + r6
    return total_reward

## Problem 2 [20]

### Iteration 1:
#### 1. Show the initial value function (V) for each state.
|   |   |
|---|---|
| 0 | 0 |
| 0 | 0 |

####  2. Perform value function updates.
Bellman Equation: $ V(s)=R(s)+γ⋅V(s')$
<br>
Due to **Initial Policy (π)** always go **up**, and default γ = 1:
- V(s1) = R(s₁) + V₀(s₁) = 5 + 0 = 5
- V(s2) = R(s₂) + V₀(s₂) = 10 + 0 = 10
- V(s3) = R(s₃) + V₀(s₁) = 1 + 0 = 1
- V(s4) = R(s₄) + V₀(s₂) = 2 + 0 = 2


####  3. Show the updated value function.
|   |   |
|---|---|
| 5 | 10 |
| 1 | 2 |


### Iteration 2:
#### Show the value function (V) after the second iteration.
- V₂(s₁) = R(s₁) + V₁(s₁) = 5 + 5 = 10
- V₂(s₂) = R(s₂) + V₁(s₂) = 10 + 10 = 20
- V₂(s₃) = R(s₃) + V₁(s₁) = 1 + 5 = 6
- V₂(s₄) = R(s₄) + V₁(s₂) = 2 + 10 = 12

|   |   |
|---|---|
| 10 | 20 |
| 6 | 12 |

## Problem 3 [35]
### Task1: Update MDP Code
#### 1. Update the reward function to be a list of reward based on whether the state is terminal, grey, or a regular state.
- updated the `value_iteration.py` below.

In [ ]:
#!/usr/bin/python3

import numpy as np

ENV_SIZE = 5


class GridWorld():

    def __init__(self, env_size):
        self.env_size = env_size
        # Initialize the value function and set terminal state value to 0
        self.V = np.zeros((env_size, env_size))
        self.terminal_state = (4, 4)
        self.V[self.terminal_state] = 0

        # Define the transition probabilities and rewards
        self.actions = [(0, 1), (1, 0), (1, 0), (-1, 0)]  # Right, Down, Down, Up
        self.action_description = ["Right", "Down", "Down", "Up"]
        self.gamma = 1.0  # Discount factor

        # Updated reward function (Task 1)
        self.rewards = {
            (4, 4): 10,  # Terminal state
            (2, 2): -5,  # Grey states
            (3, 0): -5,
            (0, 4): -5
        }
        self.default_reward = -1  # Regular states

        self.pi_greedy = np.zeros((self.env_size, self.env_size), dtype=int)
        self.pi_str = [["" for _ in range(env_size)]
                       for _ in range(env_size)]  # Initialize pi_str

    '''@brief Returns the reward for a given state
    '''
    def get_reward(self, i, j):
        return self.rewards.get((i, j), self.default_reward)

    '''@brief Checks if the change in V is less than preset threshold
    '''
    def is_done(self, new_V):
        delta = abs(self.V - new_V)
        max_delta = delta.max()
        return max_delta <= 0.0001  # Theta threshold for convergence

    '''@brief Returns True if the state is a terminal state
    '''
    def is_terminal_state(self, i, j):
        return (i, j) == self.terminal_state

    '''
    @brief Overwrites the current state-value function with a new one
    '''
    def update_value_function(self, V):
        self.V = np.copy(V)

    '''
    @brief Returns the full state-value function V_pi
    '''
    def get_value_function(self):
        return self.V

    '''@brief Returns the stored greedy policy
    '''
    def get_policy(self):
        return self.pi_greedy

    '''@brief Prints the policy using the action descriptions
    '''
    def print_policy(self):
        for row in self.pi_str:
            print(
                " ".join([f"{action:7}" if action else "term   " for action in row]))

    '''@brief Calculate the maximum value by following a greedy policy
    '''
    def calculate_max_value(self, i, j):
        # Find the maximum value for the current state using Bellman's equation
        max_value = float('-inf')
        best_action = None
        best_actions_str = ""
        for action_index in range(len(self.actions)):
            next_i, next_j = self.step(action_index, i, j)
            if self.is_valid_state(next_i, next_j):
                reward = self.get_reward(i, j)  # Reward based on current state
                value = reward + self.gamma * self.V[next_i, next_j]
                if value > max_value:
                    max_value = value
                    best_action = action_index
                    best_actions_str = self.action_description[action_index]
                elif value == max_value:
                    best_actions_str += "|" + \
                        self.action_description[action_index]
        return max_value, best_action, best_actions_str

    '''@brief Returns the next state given the chosen action and current state
    '''
    def step(self, action_index, i, j):
        # Deterministic transitions: P(s'|s) = 1.0 for valid moves, else stay
        action = self.actions[action_index]
        next_i, next_j = i + action[0], j + action[1]
        if not self.is_valid_state(next_i, next_j):
            next_i, next_j = i, j  # Stay in place if invalid
        return next_i, next_j

    '''@brief Checks if a state is within the acceptable bounds of the environment
    '''
    def is_valid_state(self, i, j):
        valid = 0 <= i < self.env_size and 0 <= j < self.env_size
        return valid

    def update_greedy_policy(self):
        for i in range(ENV_SIZE):
            for j in range(ENV_SIZE):
                if self.is_terminal_state(i, j):
                    self.pi_greedy[i, j] = 0  # No action in terminal state
                    self.pi_str[i][j] = ""
                    continue
                _, self.pi_greedy[i, j], action_str = self.calculate_max_value(
                    i, j)
                self.pi_str[i][j] = action_str


# Perform Value Iteration
gridworld = GridWorld(ENV_SIZE)
num_iterations = 1000

for _ in range(num_iterations):
    new_V = np.copy(gridworld.get_value_function())
    for i in range(ENV_SIZE):
        for j in range(ENV_SIZE):
            if not gridworld.is_terminal_state(i, j):
                new_V[i, j], _, _ = gridworld.calculate_max_value(i, j)
    gridworld.update_value_function(new_V)

# Print results
print("Optimal Value Function:")
print(gridworld.get_value_function())

print("\nOptimal Policy:")
gridworld.update_greedy_policy()
gridworld.print_policy()


####  2. Run the existing code developed in class and obtain the optimal state-values and optimal policy. Provide a figures of the gridworld with the obtained V∗ and π∗ (You can manually create a table)
![Task1: optimal state-values and optimal policy](https://github.com/Thuang6413/ConestogaCollegeWorkplaceLevel2/blob/main/CSCN8020-25S-Sec1-RLP/Assignments/Problem3Task1.png?raw=true)

### Task2: 
####  1. In-Place Value Iteration: Use a single array to store the state values. This means that you update the value of a state and immediately use that updated value in the subsequent updates.
- updated the `value_iteration.py` below.

In [ ]:
import time  # For measuring optimization time

################################################################################    
###------- Stay same code as above, but with in-place value iteration -------###
################################################################################

def is_done(self, delta, theta_threshold):
    return delta <= theta_threshold  # Theta threshold for convergence

################################################################################    
###------- Stay same code as above, but with in-place value iteration -------###
################################################################################

# Perform In-Place Value Iteration
gridworld = GridWorld(ENV_SIZE)
num_iterations = 1000
theta_threshold = 0.0001 # As a standard convergence threshold in reinforcement

start_time = time.time()  # Start timing
for iter in range(num_iterations):
    delta = 0
    for i in range(ENV_SIZE):
        for j in range(ENV_SIZE):
            if not gridworld.is_terminal_state(i, j):
                v_old = gridworld.V[i, j]
                gridworld.V[i, j], _, _ = gridworld.calculate_max_value(i, j)
                delta = max(delta, abs(v_old - gridworld.V[i, j]))
    if gridworld.is_done(delta, theta_threshold):
        break
runtime = time.time() - start_time  # End timing

# Print results
print("In-Place Value Iteration")
print(f"Optimization Time: {runtime:.6f} seconds")
print(f"Number of Iterations: {iter + 1}")
print("\nOptimal Value Function (after %d iterations):" % (iter + 1))
for i in range(ENV_SIZE):
    row = [f"{gridworld.V[i, j]:7.2f}" for j in range(ENV_SIZE)]
    print(" ".join(row))

print("\nOptimal Policy:")
gridworld.update_greedy_policy()
gridworld.print_policy()

![Task2: optimal state-values and optimal policy](https://github.com/Thuang6413/ConestogaCollegeWorkplaceLevel2/blob/main/CSCN8020-25S-Sec1-RLP/Assignments/Problem3Task2.png?raw=true)

### Comments on Complexity
- **Time**: Both have identical asymptotic time complexity. In-Place is significantly faster in practice due to:
  - Fewer iterations (9 vs. 1000).
  - No array copying, reduces memory.
- **Duplicate Actions**: The action set `["Right", "Down", "Down", "Up"]` includes redundant “Down” actions, increasing computation slightly. This affects both algorithms equally.


## Problem 4 [35]
### Task
Implement the off-policy Monte Carlo with Importance sampling algorithm to estimate the value function for the given gridworld. Use a fixed behavior policy b(a|s) (e.g., a random policy) to generate episodes and a greedy target policy.

In [ ]:
#!/usr/bin/python3

import numpy as np
import time  # For measuring optimization time

ENV_SIZE = 5


class GridWorld:
    def __init__(self, env_size):
        self.env_size = env_size
        # Initialize the value function and set terminal state value to 0
        self.V = np.zeros((env_size, env_size))
        # For importance sampling weights
        self.C = np.zeros((env_size, env_size))
        self.terminal_state = (4, 4)
        self.V[self.terminal_state] = 0  # Terminal state value

        # Define actions and rewards
        self.actions = [(0, 1), (1, 0), (1, 0), (-1, 0)]  # Right, Down, Down, Up
        self.action_description = ["Right", "Down", "Down", "Up"]
        self.gamma = 0.9  # Discount factor (Problem 4)

        self.rewards = {
            (4, 4): 10,  # Terminal state
            (2, 2): -5,  # Grey states
            (3, 0): -5,
            (0, 4): -5
        }
        self.default_reward = -1  # Regular states

        self.pi_greedy = np.zeros((self.env_size, self.env_size), dtype=int)
        self.pi_str = [["" for _ in range(env_size)] for _ in range(env_size)]

    '''@brief Returns the reward for a given state
    '''
    def get_reward(self, i, j):
        return self.rewards.get((i, j), self.default_reward)

    '''@brief Returns True if the state is a terminal state
    '''
    def is_terminal_state(self, i, j):
        return (i, j) == self.terminal_state

    '''@brief Returns the full state-value function V_pi
    '''
    def get_value_function(self):
        return self.V

    '''@brief Returns the stored greedy policy
    '''
    def print_policy(self):
        for row in self.pi_str:
            print(
                " ".join([f"{action:7}" if action else "term   " for action in row]))

    '''@brief Calculate the maximum value by following a greedy policy
    '''
    def calculate_max_value(self, i, j):
        max_value = float('-inf')
        best_action = None
        best_actions_str = ""
        best_actions = []
        for action_index in range(len(self.actions)):
            next_i, next_j = self.step(action_index, i, j)
            if self.is_valid_state(next_i, next_j):
                reward = self.get_reward(i, j)
                value = reward + self.gamma * self.V[next_i, next_j]
                if value > max_value - 1e-10:  # Handle numerical precision
                    if value > max_value + 1e-10:
                        max_value = value
                        best_actions = [action_index]
                        best_actions_str = self.action_description[action_index]
                    else:
                        best_actions.append(action_index)
                        best_actions_str += "|" + \
                            self.action_description[action_index]
        if best_actions:
            best_action = best_actions[0]  # Choose first optimal action
        return max_value, best_action, best_actions_str, best_actions

    '''@brief Returns the next state given the chosen action and current state
    '''
    def step(self, action_index, i, j):
        action = self.actions[action_index]
        next_i, next_j = i + action[0], j + action[1]
        if not self.is_valid_state(next_i, next_j):
            next_i, next_j = i, j  # Stay if invalid
        return next_i, next_j

    '''@brief Checks if a state is within the acceptable bounds of the environment
    '''
    def is_valid_state(self, i, j):
        valid = 0 <= i < self.env_size and 0 <= j < self.env_size
        return valid

    '''Return probability of each action under random behavior policy.
    '''
    def behavior_policy(self):
        return 1.0 / len(self.actions)  # Uniform: 1/4 = 0.25

    '''Return π(a|s) for action at state (i,j).
    '''
    def target_policy_prob(self, action_index, i, j):
        _, _, _, best_actions = self.calculate_max_value(i, j)
        if action_index in best_actions:
            # Split probability among optimal actions
            return 1.0 / len(best_actions)
        return 0.0

    '''Generate an episode using behavior policy.
    '''
    def generate_episode(self):
        episode = []
        i, j = 0, 0  # Start at (0,0)
        while not self.is_terminal_state(i, j):
            action_index = np.random.choice(len(self.actions))  # Random action
            next_i, next_j = self.step(action_index, i, j)
            reward = self.get_reward(i, j)  # Reward for current state
            episode.append((i, j, action_index, reward))
            i, j = next_i, next_j
        # Add terminal state with its reward
        terminal_reward = self.get_reward(i, j)
        episode.append((i, j, None, terminal_reward))
        return episode

    '''Update greedy policy based on current value function.
    '''
    def update_greedy_policy(self):
        for i in range(ENV_SIZE):
            for j in range(ENV_SIZE):
                if self.is_terminal_state(i, j):
                    self.pi_greedy[i, j] = 0
                    self.pi_str[i][j] = ""
                    continue
                _, self.pi_greedy[i, j], action_str, _ = self.calculate_max_value(
                    i, j)
                self.pi_str[i][j] = action_str

    '''Run off-policy Monte Carlo with importance sampling.
    '''
    def monte_carlo_off_policy(self, num_episodes):
        for episode_idx in range(num_episodes):
            episode = self.generate_episode()
            G = 0  # Return
            W = 1.0  # Importance sampling ratio
            # Process episode backward
            for t in range(len(episode) - 1, -1, -1):  # Include terminal state
                i, j, action_index, reward = episode[t]
                G = reward + self.gamma * G  # Update return
                if t < len(episode) - 1:  # Skip ratio for terminal action
                    W *= self.target_policy_prob(action_index,
                                                 i, j) / self.behavior_policy()
                self.C[i, j] += W
                if self.C[i, j] > 0:
                    self.V[i, j] += (W / self.C[i, j]) * (G - self.V[i, j])


# Run Off-policy Monte Carlo
gridworld = GridWorld(ENV_SIZE)
num_episodes = 10000

start_time = time.time()  # Start timing
gridworld.monte_carlo_off_policy(num_episodes)
runtime = time.time() - start_time  # End timing

# Print results
print("Off-policy Monte Carlo with Importance Sampling")
print(f"Optimization Time: {runtime:.6f} seconds")
print(f"Number of Episodes: {num_episodes}")
print("\nEstimated Value Function:")
for i in range(ENV_SIZE):
    row = [f"{gridworld.V[i, j]:7.2f}" for j in range(ENV_SIZE)]
    print(" ".join(row))

print("\nGreedy Policy:")
gridworld.update_greedy_policy()
gridworld.print_policy()


![Problem4: optimal state-values and optimal policy](https://github.com/Thuang6413/ConestogaCollegeWorkplaceLevel2/blob/main/CSCN8020-25S-Sec1-RLP/Assignments/Problem4Task.png?raw=true)

### Detailed Comparison
- **Optimization Time:** In-Place Value Iteration is significantly faster than Off-policy Monte Carlo. This significant difference arises because Value Iteration performs deterministic updates over all 9 iterations, while Monte Carlo processes 10,000 episodes.
- **Complexity:** Monte Carlo’s scales with the number of episodes and step length, making it less efficient for this small MDP but adaptable to unknown environments.
- **Model Dependence:** In-Place Value Iteration requires a known model. However, Monte Carlo is Model-free that relys on sampled episodes, suitable for scenarios where the model is unavailable.